In [1]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_pickle("/content/drive/MyDrive/transcript_texts.pkl")
df.head(5)

,Company,Call,Date_of_call,Type_of_text,Person,Text,Order,Industry
0,Abbott Laboratories,"Abbott Laboratories, Q1 2018 Earnings Call, Ap...",2018-04-18,Presentation Operator Message,Operator,Good morning and thank you for standing by. We...,0,Healthcare
1,Abbott Laboratories,"Abbott Laboratories, Q1 2018 Earnings Call, Ap...",2018-04-18,Presenter Speech,Scott Leinenweber,Good morning and thank you for joining us. \r\...,1,Healthcare
2,Abbott Laboratories,"Abbott Laboratories, Q1 2018 Earnings Call, Ap...",2018-04-18,Presenter Speech,Miles White,"Okay. Thanks, Scott, and good morning. \r\nTod...",2,Healthcare
3,Abbott Laboratories,"Abbott Laboratories, Q1 2018 Earnings Call, Ap...",2018-04-18,Presenter Speech,Brian Yoor,"Okay. Thanks, Miles. \r\nAnd as Scott mentione...",3,Healthcare
4,Abbott Laboratories,"Abbott Laboratories, Q1 2018 Earnings Call, Ap...",2018-04-18,Question and Answer Operator Message,Operator,[Operator Instructions] And our first question...,4,Healthcare


In [4]:
df['Date_of_call'].nunique()

385

In [5]:
# filter df based on the date on 'Date_of_call' column for values only from 2020 onwards

df = df[df['Date_of_call'] >= '2021-01-01']

In [6]:
df['Date_of_call'].nunique()

126

In [7]:
df[df['Industry']=='Semiconductor']['Company'].unique()

array(['Advanced Micro Devices, Inc.', 'Intel Corporation',
       'Micron Technology, Inc.', 'NVIDIA Corporation',
       'Texas Instruments Incorporated'], dtype=object)

In [8]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [9]:
model = genai.GenerativeModel('gemini-pro')

In [10]:
grouped_df = df.groupby('Call')['Text'].apply(lambda x: ' '.join(x))

grouped_df.head(5)

Call
Abbott Laboratories, Q1 2021 Earnings Call, Apr 20, 2021    Good morning, and thank you for standing by. W...
Abbott Laboratories, Q1 2022 Earnings Call, Apr 20, 2022    Good morning, and thank you for standing by. W...
Abbott Laboratories, Q2 2021 Earnings Call, Jul 22, 2021    Good morning, and thank you for standing by. W...
Abbott Laboratories, Q3 2021 Earnings Call, Oct 20, 2021    Good morning, and thank you for standing by. W...
Abbott Laboratories, Q4 2020 Earnings Call, Jan 27, 2021    Good morning, and thank you for standing by. W...
Name: Text, dtype: object

In [11]:
grouped_df.shape[0]

242

In [12]:
# batch_size = len(grouped_df.values[0].split("."))//15
# values = grouped_df.values[0].split(".")

In [13]:
# counter = 0
# res = []
# tmp = ""
# for val in values:
#   tmp += val
#   counter += 1
#   if counter == batch_size:
#     res.append(tmp)
#     tmp = ""
#     counter = 0

In [14]:
from tqdm.notebook import tqdm

In [15]:
overall = []
for transcript in tqdm(grouped_df.values):
  values = transcript.split(".")
  batch_size = len(values)//30
  for batch in values:
    counter = 0
    res = []
    tmp = ""
    for val in values:
      tmp += val
      counter += 1
      if counter == batch_size:
        res.append(tmp)
        tmp = ""
        counter = 0
    if len(tmp) != 0:
      res.append(tmp)
      tmp = ""
      counter = 0

  overall.append(res)

  0%|          | 0/242 [00:00<?, ?it/s]

In [ ]:
# overall[0][0]

In [24]:
def get_sentiment(transcript):
  # print("Working on ",transcript)
  prompt = transcript + """
  \nBased on the batch of company's earnings call transcript above, classify the company's current and forward outlook either Positive, Negative or Neutral. If there is even a slight negative sentiment, please classify it as Negative. Must take note to respond only with one word of Positive or Negative or Neutral, no other responses.
  """
  response = model.generate_content(prompt)
  return response.text

In [17]:
get_sentiment(overall[197][0]), print(overall[197][0])

Thank you for standing by Good day, everyone, and welcome to the Boeing Company's Second Quarter 2021 Earnings Conference Call Today's call is being recorded The management discussion and slide presentation, plus the analyst question-and-answer session, are being broadcast live over the Internet [Operator Instructions] At this time, for opening remarks and introductions, I'm turning the call over to Ms Maurita Sutedja, Vice President of Investor Relations for the Boeing Company Ms Sutedja, please go ahead Thank you, John Thank you, and good morning Welcome to Boeing's Second Quarter 2021 Earnings Call I'm Maurita Sutedja, and with me today are David Calhoun, Boeing's President and Chief Executive Officer; and Dave Dohnalek, Boeing's interim Chief Financial Officer And as a reminder, you can follow today's broadcast and slide presentation through our website at boeingcom  
As always, we have provided detailed financial information in our press release issued earlier today Projections, e

('Neutral', None)

In [ ]:
# print(overall[0][0])

In [18]:
# Change start and end based on assigned range

start = 1 # @param {type:"integer"}
end = 3 # @param {type:"integer"}

In [25]:
import time
final_res = {}
for i,transcript in tqdm(enumerate(overall[start:end],1)):
  for j,batch in tqdm(enumerate(transcript,1)):
    if j % 5 == 0:
      time.sleep(5)
    try:
      sentiment = get_sentiment(batch)
    except:
      print("Failed for transcript ",i," and batch ",j)
      continue
    final_res[(i, j)] = {'Text': batch, 'Sentiment': sentiment}
print(final_res)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 7716.80ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2499.36ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1969.52ms


0it [00:00, ?it/s]

ERROR:tornado.access:503 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2269.63ms


{(1, 1): {'Text': "Good morning, and thank you for standing by Welcome to Abbott's First Quarter 2022 Earnings Conference Call [Operator Instructions] This call is being recorded by Abbott With the exception of any participant's questions asked during the question-and-answer session, the entire call, including the question-and-answer session, is material copyrighted by Abbott It cannot be recorded or rebroadcast without Abbot's expressed written permission  \r\nI would now like to introduce Mr Scott Leinenweber, Vice President, Investor Relations, Licensing and Acquisitions Good morning, and thank you for joining us With me today are Robert Ford, Chairman and Chief Executive Officer; and Bob Funck, Executive Vice President, Finance and Chief Financial Officer Robert and Bob will provide opening remarks Following their comments, we will take your questions\r\nBefore we get started, some statements made today may be forward-looking for purposes of the Private Securities Litigation Reform

In [20]:
t1= 1
sent_stored = {}
temp = []
for key,value in final_res.items():
  # print("working on ")
  trans= key[0]
  if trans == t1:
    temp.append(value)
  else:
    sent_stored[t1] = temp
    t1 = trans
    temp = []

In [21]:
print(final_res)

{(1, 1): {'Text': "Good morning, and thank you for standing by Welcome to Abbott's First Quarter 2022 Earnings Conference Call [Operator Instructions] This call is being recorded by Abbott With the exception of any participant's questions asked during the question-and-answer session, the entire call, including the question-and-answer session, is material copyrighted by Abbott It cannot be recorded or rebroadcast without Abbot's expressed written permission  \r\nI would now like to introduce Mr Scott Leinenweber, Vice President, Investor Relations, Licensing and Acquisitions Good morning, and thank you for joining us With me today are Robert Ford, Chairman and Chief Executive Officer; and Bob Funck, Executive Vice President, Finance and Chief Financial Officer Robert and Bob will provide opening remarks Following their comments, we will take your questions\r\nBefore we get started, some statements made today may be forward-looking for purposes of the Private Securities Litigation Reform

In [27]:
data1=[]

for key, value in final_res.items():
  data1.append({'batch_number':key, 'Text': value['Text'], 'Sentiment': value['Sentiment']})
df_res = pd.DataFrame(data1)
df_res.head(2)

,batch_number,Text,Sentiment
0,"(1, 1)","Good morning, and thank you for standing by We...",Neutral
1,"(1, 2)",com Note that Abbott has not provided the GAAP...,Positive


In [28]:
df_res.Sentiment.unique()

array(['Neutral', 'Positive', 'Negative'], dtype=object)

In [29]:
# One transcription per row to give overall sentiment
# from collections import defaultdict
# Create a defaultdict to store counts of each sentiment for each transcript number
# transcript_counts = defaultdict(lambda: {'Positive': 0, 'Neutral': 0, 'Negative': 0})

# Count the sentiments for each transcript number
# for (transcript_number, _), sentiment in final_res.items():
    # Check if sentiment is one of the expected values before incrementing the count
    # if sentiment in ('Positive', 'Neutral', 'Negative'):
        # transcript_counts[transcript_number][sentiment] += 1

# Determine the majority sentiment for each transcript number
# majority_sentiments = {}
# for transcript_number, counts in transcript_counts.items():
    # majority_sentiment = max(counts, key=counts.get)
    # majority_sentiments[transcript_number] = majority_sentiment

# Print or use majority_sentiments as needed
# for transcript_number, sentiment in majority_sentiments.items():
    # print(f"Transcript {transcript_number}: Overall Sentiment - {sentiment}")

In [30]:
# print(majority_sentiments)

In [31]:
# majority_sentiment_df = pd.DataFrame.from_dict(majority_sentiments, orient='index', columns=['Overall Sentiment'])

In [32]:
grouped_df.head(2)

Call
Abbott Laboratories, Q1 2021 Earnings Call, Apr 20, 2021    Good morning, and thank you for standing by. W...
Abbott Laboratories, Q1 2022 Earnings Call, Apr 20, 2022    Good morning, and thank you for standing by. W...
Name: Text, dtype: object

In [33]:
grouped_df2 = grouped_df.to_frame(name='Text')
grouped_df2 = grouped_df2.iloc[start:end]
grouped_df2.reset_index(inplace=True)
grouped_df2.index = grouped_df2.index + 1
grouped_df2

,Call,Text
1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","Good morning, and thank you for standing by. W..."
2,"Abbott Laboratories, Q2 2021 Earnings Call, Ju...","Good morning, and thank you for standing by. W..."


In [34]:
# df_res['batch_number'].unique()

In [35]:
final_df = pd.merge(grouped_df2['Call'], df_res, left_index=True, right_on=df_res['batch_number'].apply(lambda x: x[0]))
final_df.head(5)

,key_0,Call,batch_number,Text,Sentiment
0,1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","(1, 1)","Good morning, and thank you for standing by We...",Neutral
1,1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","(1, 2)",com Note that Abbott has not provided the GAAP...,Positive
2,1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","(1, 3)","\r\nTurning to Nutrition, where our performa...",Negative
3,1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","(1, 4)","\r\nMoving to Diagnostics, where sales grew ...",Positive
4,1,"Abbott Laboratories, Q1 2022 Earnings Call, Ap...","(1, 5)",S to treat more patients suffering from earlie...,Positive


In [36]:
# final_df['Sentiment'] = final_df.index.map(lambda x: {k[1]: v for k, v in final_res.items() if k[0] == x})

In [37]:
final_df[['Company','Call','Date']] = final_df['Call'].str.split(',',2,expand=True)
final_df = final_df[['Call','Company','Date','batch_number','Text','Sentiment']]
final_df.head(2)

<ipython-input-37-6e98bfc02026>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  final_df[['Company','Call','Date']] = final_df['Call'].str.split(',',2,expand=True)


,Call,Company,Date,batch_number,Text,Sentiment
0,Q1 2022 Earnings Call,Abbott Laboratories,"Apr 20, 2022","(1, 1)","Good morning, and thank you for standing by We...",Neutral
1,Q1 2022 Earnings Call,Abbott Laboratories,"Apr 20, 2022","(1, 2)",com Note that Abbott has not provided the GAAP...,Positive


In [38]:
final_df.to_excel('/content/drive/MyDrive/text_transcripts_trial.xlsx')